## Invisible Cloaking App

In [87]:
import cv2
import numpy as np
import time

In [88]:
# Function to capture the background image
def capture_background(cap, num_frames=60):
    for _ in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            continue
    return frame

In [89]:
# Function to create a mask for a specific color and include everything within the shape
def create_mask(hsv, lower_bound, upper_bound):
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))

    # Find contours and fill them
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)

    return mask, contours

In [90]:
# Function to replace the colored areas with the background
def apply_cloaking(frame, background, mask):
    mask_inv = cv2.bitwise_not(mask)
    background_part = cv2.bitwise_and(background, background, mask=mask)
    frame_part = cv2.bitwise_and(frame, frame, mask=mask_inv)
    return cv2.addWeighted(background_part, 1, frame_part, 1, 0)

In [91]:
def main():
    # Open the camera
    cap = cv2.VideoCapture(0)
    
    # Inform the user and wait for 5 seconds
    print("Capturing background in 5 seconds. Please move away from the camera.")
    time.sleep(5)
    
    # Capture the background image
    print("Capturing background. Please stand still.")
    background = capture_background(cap)
    
    # Define the color range for cloaking (strong blue color)
    lower_bound = np.array([100, 100, 100])
    upper_bound = np.array([140, 255, 255])
    
    masking_on = True  # Flag to control masking
    
    print("Processing frames. Press 'q' to quit or 'x' to toggle masking.")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        if masking_on:
            # Create a mask for the specified color
            mask, contours = create_mask(hsv, lower_bound, upper_bound)
            
            # Apply the cloaking effect
            cloaked_frame = apply_cloaking(frame, background, mask)
            
            # Draw contours of the detected object (optional)
            cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)
        else:
            cloaked_frame = frame
        
        # Display the result
        cv2.imshow('Cloaker', cloaked_frame)
        
        # Keyboard input handling
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('x'):
            masking_on = not masking_on
            if masking_on:
                print("Masking turned ON")
            else:
                print("Masking turned OFF")
    
    cap.release()
    cv2.destroyAllWindows()

In [93]:
if __name__ == "__main__":
    main()

Capturing background in 5 seconds. Please move away from the camera.
Capturing background. Please stand still.
Processing frames. Press 'q' to quit or 'x' to toggle masking.
Masking turned OFF
Masking turned ON
